# Objectif : Prédire la  constitution du dernier panier d'achat éffectuée par les consommateurs 

# on va choisir de travailler avec 12000 clients 

In [ ]:
# import des librairies 
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM
import numpy as np 
import pandas as pd 



Using TensorFlow backend.


# Chargement des données de nos 200 premiers clients 

In [ ]:
def read_orders(N ,path="C:/Users/Asus/Desktop/machine learning/4931_7487_compressed_orders.csv"):
    df_orders = pd.read_csv(path)
    df_orders =df_orders[['order_id','user_id']]
    return df_orders[df_orders['user_id'].isin(range(N+1))]

def read_orders_products(path="C:/Users/Asus/Desktop/machine learning/4931_7487_compressed_order_products__prior.csv"):
    order_product_df=pd.read_csv(path)
    return order_product_df[['order_id','product_id']]
products =pd.read_csv("C:/Users/Asus/Desktop/machine learning/4931_7487_compressed_products.csv")

In [ ]:
# on definit le nombre de clients
n_client= 12000
order_df =read_orders(n_client)
order_product_df =read_orders_products()

# Réalisation des jointures sql 

In [ ]:
# Prémière jointure sur la clé order_id 
data_first_fusion= pd.merge(order_df,order_product_df , on ='order_id')

In [ ]:
# lecture des produits dans product_df
products_df=products[['product_id','department_id']]


In [ ]:
# deuxieme jointure sur la clé product_id 
data_second_fusion=pd.merge(data_first_fusion,products_df , on ='product_id')

In [ ]:
#print(data_second_fusion)
data_final= data_second_fusion[['order_id','user_id','department_id']]
print(data_final)

order_id  user_id  department_id
0         2539329        1              7
1         2398795        1              7
2          473747        1              7
3         2254736        1              7
4          431534        1              7
5         3367565        1              7
6          550135        1              7
7         3108588        1              7
8         2295261        1              7
9         2550362        1              7
10        2968173       15              7
11        1870022       15              7
12        1911383       15              7
13        2715276       15              7
14         487368       15              7
15        2293453       19              7
16        1973799       19              7
17         532817       19              7
18        1573906       21              7
19        1593000       31              7
20        2231262       31              7
21        2580647       43              7
22        2187180       43              7
2

# Procédure pour recuperer les les paniers de tous nos 200 clients 

In [ ]:
exemple=[]
for client in data_final.groupby("user_id"):
    client_id=client[0]  # on va recup id_client
    client_ex=client[1]  # on conserve ici les order_id et department
    
    Basket=[]    # cette variable va stocker le panier client 
    for order in client_ex.groupby("order_id"):
        order_id=order[0]   # on va recup l'order_id
        order_ex=order[1]   # on va conserver le dpt_id
        dpt_id=list(order_ex["department_id"])
        
        Basket.append(dpt_id)
    exemple.append(Basket)
print(exemple)

, 16], [7, 19, 4, 12, 12, 9, 12, 11, 19, 3, 15, 7, 19, 3, 13, 16], [7, 19, 12, 9, 13, 1, 12, 1, 19, 9, 16, 1, 3, 13, 13, 16], [20, 7, 19, 1, 19, 15, 1, 12, 16, 7, 13, 1, 1, 7, 3, 7], [4, 12, 9, 17, 17], [7, 7, 1, 19, 19], [7, 1, 4, 1, 9, 7, 1, 12, 11, 13, 19, 9, 16, 1, 7, 15, 7, 17, 13, 16], [7, 19, 12, 4, 1, 19, 9, 1, 13, 16], [7, 13, 19, 4, 16, 19, 12, 19, 1, 12, 11, 13, 11, 13, 1, 19, 16, 14, 13, 15, 17, 13, 16], [19, 1, 17, 16], [19, 19, 9, 16, 13], [7, 19, 1, 4, 16, 16, 1, 12, 1, 1, 16, 1, 1, 15, 3, 16], [4, 19, 16, 4, 1, 1, 12, 11, 19, 9, 16, 7, 16, 13], [4, 11, 19, 3], [7, 4, 12, 7, 1, 19, 16, 3, 16], [7, 1, 19, 12, 12, 11, 16, 14, 7, 15, 16], [7, 19, 4, 12, 1, 12, 7, 1, 12, 17, 16, 15, 17, 13], [7, 19, 16, 4, 16, 12, 1, 12, 1, 15, 7, 19], [4, 19, 9, 16, 13], [20, 7, 19, 4, 9, 16, 1, 12, 19, 1, 1, 1, 1, 17, 13, 16], [7, 19, 16, 12, 4, 1, 6, 19, 12, 13, 11, 17, 9, 13, 17, 9, 19, 16, 1, 14, 3, 15, 17, 13, 16], [7, 19, 4, 16, 16, 16, 1, 12, 19], [7, 19, 1, 4, 1, 15, 1, 12, 11, 17, 

# initialisation du Pad_Sequence : principe consistant a équilibrer notre matrice a la meme taille 

In [ ]:

from keras.preprocessing.sequence import pad_sequences

c=8 # on va limiter nos dpt par basket a 8 
t=3 # ici on va limiter le nombre de panier client a 3
Z=np.zeros((12000,3,8)) 

for i in range(len(exemple)):
    xi=pad_sequences((exemple[i]), maxlen=c, dtype='int32', padding='post', truncating='post', value=0.0)
    t=min(t,xi.shape[0])
    xi=xi[:t]
    Z[i] =xi
print(Z)


[[[ 7. 19. 19. ...  4.  4.  4.]
  [ 7. 19. 19. ...  0.  0.  0.]
  [ 7. 19. 19. ...  0.  0.  0.]]

 [[19. 20. 20. ...  4. 19.  4.]
  [19. 19.  7. ... 19. 16. 16.]
  [19. 16.  4. ... 16. 16. 16.]]

 [[16.  4. 19. ... 16. 17.  4.]
  [ 4. 16. 16. ...  0.  0.  0.]
  [ 4. 16. 16. ...  7.  0.  0.]]

 ...

 [[13.  4.  1. ...  1. 16.  1.]
  [15. 16. 16. ...  1.  4.  0.]
  [ 4.  4.  4. ... 16.  7.  4.]]

 [[ 4.  4.  4. ...  4.  4.  4.]
  [ 4.  4.  4. ...  4.  4.  4.]
  [ 4.  4.  4. ...  4. 16.  4.]]

 [[19. 19.  4. ...  3. 13.  0.]
  [ 7.  7.  7. ...  0.  0.  0.]
  [ 7.  7. 19. ...  0.  0.  0.]]]


# Ecriture de X et de Y : X =input et Y = Target

In [ ]:
# initialisation de nos vecteurs x et y 
X=Z[:,:-1,:]   # on va prendre les 2 premiers paniers 
print(X)
Y=Z[:,-1:,:]   # notre target sera le dernier panier de tous nos 200 clients 
print(Y)
#7. 19. 19. 14. 16.  0.  0.  0.]]

[[[ 7. 19. 19. ...  4.  4.  4.]
  [ 7. 19. 19. ...  0.  0.  0.]]

 [[19. 20. 20. ...  4. 19.  4.]
  [19. 19.  7. ... 19. 16. 16.]]

 [[16.  4. 19. ... 16. 17.  4.]
  [ 4. 16. 16. ...  0.  0.  0.]]

 ...

 [[13.  4.  1. ...  1. 16.  1.]
  [15. 16. 16. ...  1.  4.  0.]]

 [[ 4.  4.  4. ...  4.  4.  4.]
  [ 4.  4.  4. ...  4.  4.  4.]]

 [[19. 19.  4. ...  3. 13.  0.]
  [ 7.  7.  7. ...  0.  0.  0.]]]
[[[ 7. 19. 19. ...  0.  0.  0.]]

 [[19. 16.  4. ... 16. 16. 16.]]

 [[ 4. 16. 16. ...  7.  0.  0.]]

 ...

 [[ 4.  4.  4. ... 16.  7.  4.]]

 [[ 4.  4.  4. ...  4. 16.  4.]]

 [[ 7.  7. 19. ...  0.  0.  0.]]]


# Initialisation du modele LSTM (Long Short Time Memory)


# on va changer la dimensions de nos données 


In [ ]:
Y=np.reshape(Y, (12000, 8))

# Préparation de nos données 


In [ ]:
from sklearn.model_selection import train_test_split
x_train ,x_test ,y_train , y_test =train_test_split(X,Y ,test_size=0.25)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(9000, 2, 8)
(3000, 2, 8)
(9000, 8)
(3000, 8)


In [ ]:
# LSTM
#max_features=(150,2,8)
model = Sequential()
#model.add(Embedding(1600, dropout=0.2))
model.add(LSTM(256, dropout_W=0.2, dropout_U=0.2))# input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dense(8))
model.add (Dense (8, activation = 'relu')) 
#model.add(Dense(8))
model.add(Activation('softmax'))



# Compilation 

In [ ]:
# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training

In [ ]:
model.fit(x_train, y_train, batch_size=1, verbose=1, nb_epoch=10, validation_data=(x_test, y_test))

Train on 9000 samples, validate on 3000 samples
Epoch 1/10
9000/9000 [==============================] - 123s 14ms/step - loss: 14.1014 - accuracy: 0.2249 - val_loss: 14.2519 - val_accuracy: 0.2194
Epoch 2/10
9000/9000 [==============================] - 147s 16ms/step - loss: 13.9773 - accuracy: 0.2250 - val_loss: 14.1805 - val_accuracy: 0.2194
Epoch 3/10
9000/9000 [==============================] - 155s 17ms/step - loss: 13.9698 - accuracy: 0.2250 - val_loss: 14.1784 - val_accuracy: 0.2194
Epoch 4/10
9000/9000 [==============================] - 199s 22ms/step - loss: 13.9643 - accuracy: 0.2249 - val_loss: 14.1766 - val_accuracy: 0.2194
Epoch 5/10
9000/9000 [==============================] - 213s 24ms/step - loss: 13.9649 - accuracy: 0.2249 - val_loss: 14.1727 - val_accuracy: 0.2194
Epoch 6/10
9000/9000 [==============================] - 347s 39ms/step - loss: 13.9548 - accuracy: 0.2250 - val_loss: 14.1734 - val_accuracy: 0.2194
Epoch 7/10
9000/9000 [==============================] - 35

# Evaluation du model 

In [ ]:
# Evalaution
score = model.evaluate(x_test, y_test, batch_size=50)
print ("Test Metrics:"), (model.metrics_names, score,"%")
#plot(model, to_file='s8.png', show_shapes=True

3000/3000 [==============================] - 0s 124us/step
Test Metrics:


(None, (['loss', 'accuracy'], [14.150785319010417, 0.21937499940395355], '%'))